In [7]:
# --- Imports ---
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output


from mcmc import *

# Monte-Carlo Options Pricing: A Visualization
While we have included more detail in the following sections, we will begin with a brief overview of what our project aims to achieve.

<!-- TODO: This is literally just the pasted abstract we submitted with the proposal. -->
Our project addresses the challenge of understanding how uncertainty and randomness affect the price of options in finance. Pricing options requires estimating the expected future value of an asset, a task that becomes analytically intractable when the option has complex features or many sources of uncertainty. Monte Carlo simulations are a powerful technique to approximate option prices by simulating many potential future price paths, but their stochastic nature can make them difficult to grasp.

Our visualization aims to bridge this gap. Users will be able to simulate option pricing based on adjustable parameters like volatility, drift, interest rate, time to expiration, and number of simulation paths. As paths evolve visually, a convergence plot will dynamically show the option price estimate stabilizing. This will be plotted against the Black-Scholes model, a widely used formula for pricing options, highlighting the convergence of Monte Carlo simulations to the underlying price.

By making this probabilistic process tangible, our tool will help viewers develop intuition for how uncertainty propagates into financial valuations. The final deliverable will be a web-based interactive visualization combining stochastic process simulation with real-time data plots to deliver an experience that demystifies the use of Monte Carlo simulations in option pricing.

# Basics of Options Pricing
For our final project, we will be visualizing how Monte Carlo simulations are used to price options.

We'll begin with some explanations. Options are a financial contract that give the buyer the *right* (but not the obligation! hence the name "option") to buy or sell an asset at a specific price, the **strike price**, before the option's **expiration date**. There are two main kinds of options: **call options**, which let you buy an asset at the strike price before the expiration date, and **put options**, which let you sell an asset at the strike price before the expiration date.

For example, let's say you want to buy a call option with a strike price of `$110` that expires in 1 month on Pepsi stock, which is currently worth `$100`. How would you price this option? If you pay `$10` for the option, you are hoping that Pepsi stock will be worth more than `$110` in 1 month, so that you can buy it at `$110` and sell it for more than you paid for the option.

Say we buy this option for `$10`. If Pepsi stock goes up to `$135` in 1 month, you can use the option to buy it at `$110` and then sell it for `$135`, netting you a profit of `$135` (what you sell it for) - `$110` (the strike price you buy at) - `$10` (the price of the option) = `$15`. You can see that the option is now worth`$15`, so it is a good deal!

However, assume that Pepsi stock goes down to `$90` in 1 month. If you bought the option, you would not use it, because you could just buy Pepsi stock for `$90` - there's no reason to buy it at the option's strike price of `$110`. In this case, the option is worthless, and you would have lost the `$10` you paid for it.

Therefore, when buying an option, we seek to only buy it if we think that *on average* the option will return a profit above the price of buying it.

<!-- TODO: Should this be an interactive visualization of how this can go? -->

Pricing options is important for both buyers and sellers of options. For buyers, it allows them to figure out if an option is a good deal and thus whether they can make a profit on it. For sellers, it allows them to figure out how much to charge for an option, i.e. the "fair price".

# Monte Carlo Simulation with Metropolis-Hastings

TODO: Make it less jargon-y

In our Pepsi example, it's easy to see that the option is worth `$15`, so we should buy it. However, in the real world, we don't know the exact value of the option. Instead, we can use a Monte Carlo simulation to estimate the value of the option.

Let's start with a simple coin tossing example to demonstrate how a Monte Carlo simulation works.

Say we have a coin that has an unknown probability $p$ of landing heads. An intuitive way to estimate $p$ is to flip the coin many times and take the average of the number of heads. At its core, this is what a Monte Carlo simulation does: it repeatedly draws random samples from a probability distribution and then takes the average of the function evaluated at those samples.

Let's demonstrate this now using code. For demonstration purposes, we'll begin by randomly generating a probability $p$ of landing heads, between 0 and 1 - in the real world, we wouldn't know what $p$ is!

In [ ]:
# For reproducibility.
# np.random.seed(42)

# Randomly generate the true probability of landing heads.
true_p = np.random.uniform(0, 1)

# TODO: Make sure this prints to the page.
# TODO: Make this interactive? Maybe let the user select it, 
#   or have a button to randomly regenerate it.
print(f"True probability p: {true_p:.4f}")

From here, we take this coin and toss it $N$ times, and count the number of heads and tails. This can be done using a Binomial distribution, which we simulate below:

In [ ]:
# Number of times to toss our coin.
N = 1000

num_heads, num_tails = simulate_coin_flips(true_p, N)

In [ ]:
# Function to simulate coin flips and compute the running average
def simulate_coin_data(n_flips, p):
    flips = np.random.binomial(1, p, size=n_flips)
    running_avg = np.cumsum(flips) / np.arange(1, n_flips + 1)
    df = pd.DataFrame({
        'Flip Number': np.arange(1, n_flips + 1),
        'Running Average': running_avg
    })
    return df

# Create widgets for simulation parameters
n_flips_slider = widgets.IntSlider(
    value=1000, min=10, max=5000, step=10, description='Flips'
)
p_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01, description='p'
)
run_button = widgets.Button(
    description="Run Simulation", button_style='success'
)
randomize_button = widgets.Button(
    description="Randomize p", button_style='info'
)

# Widgets for animation: a Play widget linked to a slider
animation_slider = widgets.IntSlider(
    value=1, min=1, max=n_flips_slider.value, step=1, description="Current Flip"
)
play = widgets.Play(
    value=1, min=1, max=n_flips_slider.value, step=1, interval=50, description="Press play"
)
widgets.jslink((play, 'value'), (animation_slider, 'value'))

# Output widget for displaying the animated chart
animation_output = widgets.Output()

# Global variable to store simulation data
simulation_data = None

def run_simulation(b):
    """Compute simulation data and initialize the animation slider."""
    global simulation_data
    n_flips = n_flips_slider.value
    p_val = p_slider.value
    simulation_data = simulate_coin_data(n_flips, p_val)
    
    # Update the maximum values for the slider and play widget
    animation_slider.max = n_flips
    play.max = n_flips
    animation_slider.value = 1  # reset to start
    update_animation(None)      # update the chart immediately

def update_animation(change):
    """Update the Altair chart to show the running average up to the current flip."""
    if simulation_data is None:
        return
    t = animation_slider.value
    df_subset = simulation_data[simulation_data["Flip Number"] <= t]
    
    # Create a line chart for the running average
    line_chart = alt.Chart(df_subset).mark_line().encode(
        x=alt.X("Flip Number:Q", title="Number of Flips"),
        y=alt.Y("Running Average:Q", title="Proportion of Heads")
    ).properties(
        width=600,
        height=300,
        title=f"Running Average up to Flip {t} (True p = {p_slider.value:.2f})"
    )
    
    # Create a horizontal rule for the true probability
    rule = alt.Chart(pd.DataFrame({"True p": [p_slider.value]})).mark_rule(
        color='red', strokeDash=[4, 2]
    ).encode(
        y=alt.Y("True p:Q")
    )
    
    # Combine the charts
    animated_chart = line_chart + rule
    
    with animation_output:
        clear_output(wait=True)
        display(animated_chart)

def randomize_p_func(b):
    """Randomly set a new value for p."""
    new_p = np.random.rand()
    p_slider.value = new_p

# Event handlers for button and slider
run_button.on_click(run_simulation)
randomize_button.on_click(randomize_p_func)
animation_slider.observe(update_animation, names='value')

# Arranging the UI. Note I'm not super sure if this is all correct
ui = widgets.VBox([
    widgets.HBox([n_flips_slider, p_slider]),
    widgets.HBox([randomize_button, run_button]),
    widgets.HBox([play, animation_slider]),
    animation_output
])
display(ui)


However, in this case we know the exact distribution of the coin – we can just flip it ourselves – so it's easy for us to compute the probability of heads. What if we didn't have the coin, or were dealing with a more complex set of variables? In this case, we can use something called the Metropolis-Hastings algorithm to estimate the probability of an event happening - in case of options, the probability of the stock price at expiration being above the strike price.

To do this, we have to use something called rejection sampling. What this does is draw samples from a normal distribution, and then accept or reject those samples based on whether they are above or below the *target* distribution - what we expect the upper and lower bounds of the true distribution to be. For example, if we were to randomly draw a number greater than 1 from the normal distribution when trying to estimate the probability of heads, we would reject it, because we know the probability of heads being greater than 1 is 0.

In [ ]:
# Store globals in dictionary
mh_chain_df = pd.DataFrame()   # will hold the entire chain once generated
coin_data = {'p_true': 0.6, 'N': 1000, 'heads_count': 0, 'tails_count': 0}

def generate_coin_data(p_true: float, N: int):
    """
    Generate coin flip data from a Bernoulli(p_true).
    Returns heads_count, tails_count.
    """
    flips = np.random.binomial(1, p_true, size=N)
    heads = np.sum(flips)
    tails = N - heads
    return heads, tails

# Utilities for Metropolis-Hastings
def log_likelihood(p, heads, tails):
    # Add epsilon to avoid log(0)
    eps = 1e-12
    p = np.clip(p, eps, 1 - eps)
    return heads * np.log(p) + tails * np.log(1 - p)

def propose_new_state(current_p, proposal_width=0.05):
    # Propose new state using normal random walk, clipped to [0,1]
    candidate = current_p + np.random.normal(0, proposal_width)
    return np.clip(candidate, 0, 1)

def mh_chain(n_steps, heads_count, tails_count, proposal_width=0.05, start_p=0.5):
    """
    Run Metropolis-Hastings chain for 'n_steps' given coin-flip data (heads_count, tails_count).
    Returns a DataFrame with columns:
       iteration, candidate, accepted, state
    Where:
      - 'candidate' is the proposed new p
      - 'accepted' is True/False
      - 'state' is the chain's state AFTER iteration i
    """
    chain_records = []
    current_p = start_p
    current_ll = log_likelihood(current_p, heads_count, tails_count)
    
    for i in range(1, n_steps+1):
        candidate_p = propose_new_state(current_p, proposal_width)
        cand_ll = log_likelihood(candidate_p, heads_count, tails_count)
        
        log_ratio = cand_ll - current_ll
        if np.log(np.random.rand()) < log_ratio:
            # accept
            accepted = True
            current_p = candidate_p
            current_ll = cand_ll
        else:
            # reject
            accepted = False
        
        chain_records.append({
            'iteration': i,
            'candidate': candidate_p,
            'accepted': accepted,
            'state': current_p
        })
    return pd.DataFrame(chain_records)

# Plotting 
def build_mh_charts(chain_df, i, p_true, n_steps):
    """
    Build two charts side-by-side (hconcat):
      1) A horizontal histogram of accepted states up to iteration i.
      2) A trace plot of states vs. iteration, highlighting iteration i.
      
    Args:
      chain_df: DataFrame with columns ['iteration','candidate','accepted','state'].
      i: current iteration index to display.
      p_true: the "true" probability of heads (for the red reference line).
      n_steps: total number of MH steps (to fix x-axis domain).
    """
    # Subset up to iteration i
    chain_up_to_i = chain_df[chain_df['iteration'] <= i].copy()
    
    # Only accepted states for histogram
    accepted_df = chain_up_to_i[chain_up_to_i['accepted'] == True].copy()
    
    # Plot histogram of accepted p-values. Gives a visual indication for how
    # the algorithm is converging to 'figuring out' what the true p is.
    hist = (
        alt.Chart(accepted_df)
        .mark_bar(orient="horizontal")
        .encode(
            # Fix axis to [0, 1]
            y=alt.Y('state:Q', 
                    bin=alt.Bin(maxbins=20), 
                    title='p',
                    scale=alt.Scale(domain=[0, 1])
                   ),
            x=alt.X('count()', title='Count')
        )
        .properties(width=200, height=300, title="Histogram of Accepted p-values")
    )
    
    # Add red line for true p across both the histogram and the line plot
    rule_df = pd.DataFrame({'p_true': [p_true]})
    true_p_rule = (
        alt.Chart(rule_df)
        .mark_rule(color='red')
        .encode(y='p_true:Q')
    )
    
    hist_with_rule = hist + true_p_rule

    # 2) Trace plot of states vs iteration
    accepted_pts = chain_up_to_i[chain_up_to_i['accepted'] == True]
    rejected_pts = chain_up_to_i[chain_up_to_i['accepted'] == False]

    # Connect accepted states with a line
    line_accepted = alt.Chart(accepted_pts).mark_line(color='blue').encode(
        x=alt.X("iteration:Q", 
                title="Iteration", 
                scale=alt.Scale(domain=[1, n_steps])
               ),
        y=alt.Y("state:Q", title="p", 
                scale=alt.Scale(domain=[0, 1])
               ),
    )

    # Accepted points
    accepted_chart = alt.Chart(accepted_pts).mark_circle(size=60, color='blue').encode(
        x=alt.X("iteration:Q", 
                scale=alt.Scale(domain=[1, n_steps])
               ),
        y="state:Q",
        tooltip=["iteration", "state"]
    )

    # Rejected proposals
    rejected_chart = alt.Chart(rejected_pts).mark_square(size=60, color='grey').encode(
        x=alt.X("iteration:Q", 
                scale=alt.Scale(domain=[1, n_steps])
               ),
        y="candidate:Q",
        tooltip=["iteration", "candidate"]
    )

    # Highlight the current iteration i in green
    current_iter_df = chain_df[chain_df['iteration'] == i]
    current_pt_chart = alt.Chart(current_iter_df).mark_circle(size=80, color='green').encode(
        x=alt.X("iteration:Q", scale=alt.Scale(domain=[1, n_steps])),
        y="state:Q"
    )

    trace = (
        (line_accepted + accepted_chart + rejected_chart + current_pt_chart)
        .properties(width=400, height=300, title=f"MH Trace up to iteration {i}")
    )

    # Combine horizontally
    final_chart = alt.hconcat(hist_with_rule, trace)
    return final_chart



# Buttons for data generation
# TODO: Make this clearer as well.
true_p_slider = widgets.FloatSlider(
    value=0.6, min=0.0, max=1.0, step=0.01, description="True p"
)
randomize_p_button = widgets.Button(description="Randomize p", button_style="info")

n_data_slider = widgets.IntSlider(
    value=1000, min=10, max=5000, step=10, description="Data flips"
)
gen_data_button = widgets.Button(description="Generate Data", button_style="success")

# Buttons for MCMC chain generation
# TODO: Make this clearer
# TODO: Clamp to data slider value?
n_steps_slider = widgets.IntSlider(
    value=100, min=10, max=5000, step=10, description="MH steps"
)
proposal_width_slider = widgets.FloatSlider(
    value=0.05, min=0.01, max=0.2, step=0.01, description="Proposal σ"
)
run_mh_button = widgets.Button(description="Run MCMC", button_style="success")

# Define the animation slider + play button
mh_iteration_slider = widgets.IntSlider(
    value=1, min=1, max=100, step=1, description="Iteration"
)
mh_play = widgets.Play(
    value=1, min=1, max=100, step=1, interval=200, description="Press play"
)
widgets.jslink((mh_play, 'value'), (mh_iteration_slider, 'value'))

# define the output box
viz_output = widgets.Output()

# Callback functions
def randomize_p_callback(_):
    new_p = np.random.rand()
    true_p_slider.value = new_p

def generate_data_callback(_):
    # re-generate coin flips with current p_true, N
    coin_data['p_true'] = true_p_slider.value
    coin_data['N'] = n_data_slider.value
    h, t = generate_coin_data(coin_data['p_true'], coin_data['N'])
    coin_data['heads_count'] = h
    coin_data['tails_count'] = t
    with viz_output:
        clear_output(wait=True)
        print(f"Generated data with p_true={coin_data['p_true']:.2f}, N={coin_data['N']}")
        print(f"Heads={h}, Tails={t}")

def run_mh_callback(_):
    global mh_chain_df
    # run chain with user-specified steps
    n_steps = n_steps_slider.value
    proposal_width = proposal_width_slider.value
    
    # reset the iteration slider
    mh_iteration_slider.max = n_steps
    mh_iteration_slider.value = 1
    mh_play.max = n_steps
    mh_play.value = 1
    
    # run the chain
    start_p = 0.5
    mh_chain_df = mh_chain(
        n_steps,
        coin_data['heads_count'],
        coin_data['tails_count'],
        proposal_width=proposal_width,
        start_p=start_p
    )
    
    with viz_output:
        clear_output(wait=True)
        print("MH chain generated. Use the slider or play button below to animate.")

def animate_chain(change):
    """
    Update the chart to show up to iteration i in the MH chain.
    """
    if mh_chain_df.empty:
        return  # no chain to show
    
    i = mh_iteration_slider.value
    # build altair chart
    chart = build_mh_charts(mh_chain_df, i, coin_data['p_true'], n_steps_slider.value)
    
    with viz_output:
        clear_output(wait=True)
        display(chart)

# Callbacks for button clicks
randomize_p_button.on_click(randomize_p_callback)
gen_data_button.on_click(generate_data_callback)
run_mh_button.on_click(run_mh_callback)
mh_iteration_slider.observe(animate_chain, names='value')

# Layout/display boxes; messy, need to fix
ui_top = widgets.HBox([
    widgets.VBox([
        widgets.Label("Data Generation Controls:"),
        true_p_slider,
        randomize_p_button,
        n_data_slider,
        gen_data_button
    ]),
    widgets.VBox([
        widgets.Label("MCMC Controls:"),
        n_steps_slider,
        proposal_width_slider,
        run_mh_button
    ])
])

ui_bottom = widgets.HBox([
    mh_play,
    mh_iteration_slider
])

app = widgets.VBox([
    ui_top,
    ui_bottom,
    viz_output
])

display(app)

with viz_output:
    print("1) Choose/Randomize True p, set number of data flips, and click 'Generate Data'.")
    print("2) Adjust MH steps, proposal width, and click 'Run MCMC'.")
    print("3) Use slider or press play to animate chain.")



A visualization of this is shown below, where we use rejection sampling to estimate the true probability of heads using a Monte Carlo simulation:

However, if you rerun this many times, you might note that where the estimation converges to depends more on the first few flips than on the later flips. This makes intuitive sense, because when averaging out the results, the first few flips are being divided by a smaller number than the later flips are and thus each has a bigger impact on the average.

To combat this, we can discard the first portion of flips, which is called the "burn-in" period. This lets the Monte Carlo simulation figure out a reasonable average to hover around, and then try to explore from there. 

This is shown in the visualization below:

TODO: Introduce Monte Carlo simulation of *expected* value of the option, not just the probability it will return a profit or not.

TODO: Gradually introduce different variables for the user to interact with: number of path simulations, number of iterations, volatility, strike price, drift, etc.

Now we will talk about different variables that affect what happens to an option over time. The main variables we care about are as follows:

Number of Path Simulations: The number of path simulations for a Monte Carlo Simulation is the number of possible different outcomes trees we are exploring. For example, if we have one path simulation, then we are only mapping one possible outcome. If we have two path simulations, then we are looking at two possible futures, etc etc. The more simulation paths that we observe, the more data we will have to analyze, and the more complete of a picture we will have prediting the future. A simple way we can see this is with the law of large numbers, which states that the more data we have, that dataset's mean will be closer to the true mean of whatever we are sampling. So, in our example, the more paths we have, the better an estimate of the stock's future value we will have.

Number of Iterations: The number of iterations in the algorithm shows how many steps we take to refine our estimate. This is like time to expiration in options pricing — more time allows for greater price fluctuations, just like how more iterations allows for a better estimate. With few iterations, our estimate is rough, like how an option close to expiration has limited opportunities for price movement. With more iterations, we get a more accurate distribution, just as a longer time to expiration gives the stock price more chances to impact an option’s value.

In [ ]:
import numpy as np
import pandas as pd
import altair as alt

# Ensure Altair renders properly in Jupyter
alt.renderers.enable('default')

# Function to plot results
def plot_chains(num_iterations, proposal_std, num_chains):
    chains = metropolis_hastings(num_iterations, proposal_std, num_chains)

    # Convert to DataFrame for visualization
    df = pd.DataFrame({
        'Iteration': np.tile(np.arange(num_iterations), num_chains),
        'Chain Value': chains.flatten(),
        'Chain': np.repeat(np.arange(1, num_chains + 1), num_iterations)
    })

    # Altair visualization
    chart = alt.Chart(df).mark_line().encode(
        x='Iteration:Q',
        y=alt.Y('Chain Value:Q', title='Parameter Value', scale=alt.Scale(domain=[0, 1])),
        color='Chain:N'
    ).properties(
        width=600,
        height=400,
        title=f'Metropolis-Hastings Chains (Iterations={num_iterations}, Chains={num_chains})'
    )

    return chart

# Generate coin flip data
np.random.seed(42)
true_p = np.random.uniform(0, 1)  # Random true probability
N = 1000
coin_flips = np.random.binomial(1, true_p, size=N)
num_heads = np.sum(coin_flips)
num_tails = N - num_heads

# Set parameters
# Todo: Make this interactable in the interface for the User.
num_iterations = 50 # Customize number of iterations
proposal_std = 0.5
num_chains = 25  # Customize number of path simulations

# Display the result
plot_chains(num_iterations, proposal_std, num_chains).display()


# Volatility(σ):
Volatility measures how much the price of a stock fluctuates over time. When analyzing options, there are two key types of volatility: implied volatility (IV) and historical volatility (HV).

- IV reflects the market's expectation of future price fluctuations. It measures how likely a stock is to experience major price swings (in either direction) before the option expires. 

- HV measures how much a stock's price has actually fluctuated in the past over a given periold (e.g., 30 days, 60 days, or 1 year). It is calculated using the standard deviation of past returns

IV is important because it is a key factor in determining an option's price. Higher IV indicates a greater chance the stock will reach or exceed the strike price, increasing the potential profit for buyers. As a result, options on stocks with high IV tend to be more expensive than those on stable stocks. HV is important because it serves as a benchmark for IV, helping traders assess if an options IV over or under priced. In Monte Calro simulations, IV determines the likelihood of the option expiring In The Money (ITM) or not. Because options do not require you to exercise your right to buy or sell at any time, if IV is high, the stock has a greater chance of hitting the strike price before the option expires. In other words, more large fluctuations means you have a higher chance of the stock reaching a value where you can return a profit before the option expires. 

TODO: Make it so that the user can move around the data points on HV graph so the HV can be recalculated based on the new data points. Also add explanation to interaction. Also instead of simulating use real HV for some stock instead? Add reset button as well

TODO: Make a IV graph where the user can adjust the IV and the option price will be automatically calculated 

In [ ]:
np.random.seed(42)
num_days = 60  
initial_price = 100  
true_volatility = 0.25  

# Simulate stock price movements (Geometric Brownian Motion)
returns = np.random.normal(0, true_volatility / np.sqrt(252), num_days)
prices = initial_price * np.cumprod(1 + returns)

# Compute HV
window = 10  
log_returns = np.diff(np.log(prices))  
hv_series = pd.Series(log_returns).rolling(window=window).std() * np.sqrt(252) * 100  

# Trim stock price data to match HV length
aligned_days = np.arange(window, num_days)  
df_price = pd.DataFrame({"Day": aligned_days, "Value": prices[window:], "Type": "Stock Price"})
df_hv = pd.DataFrame({"Day": aligned_days, "Value": hv_series[window-1:], "Type": "Historical Volatility"})

# Combine both datasets
df_combined = pd.concat([df_price, df_hv]).dropna()

overall_hv = hv_series.mean()

# Main chart with legend
chart = alt.Chart(df_combined).mark_line().encode(
    x=alt.X("Day:Q", title="Day"),
    y=alt.Y("Value:Q", title="Stock Price / Historical Volatility (%)", scale=alt.Scale(zero=False)),
    color=alt.Color("Type:N", title="Legend", scale=alt.Scale(domain=["Stock Price", "Historical Volatility"], range=["gray", "blue"]))
).properties(
    width=700,
    height=400,
    title="Stock Price & Historical Volatility Over Time"
)

chart.display()

print(f"Overall Historical Volatility (HV): {overall_hv:.2f}%")


# Strike price (K): 

The strike price is the predetermined price at which an option holder can buy(call option) or sell (put option) the underlying asset before the expiration date. The strike price determines if an option is In The Money (ITM), At The Money(ATM), or Out of The Money (OTM)

- ITM is when the option is profitable to exercise. For call options, this is when the stock price is above the strike price. The buyer can buy the stock for a value less than what it is currently trading at and instantly sell it for a profit. For put options, this is when the stock value is below the strike price. The buyer can buy the stock for a value less than the current trading price and instantly sell them at the option price for a profit.

- ATM is when the stock price is equal to the strike price. Exercising the option results in no profit or loss , besides the premium paid for the option.

- OTM is when expercising the option would result in a loss. The option therefor has no value and is worthless.

TODO: Make it so the user can drag the data points of the stock price and the color of the line dynamically changes

TODO: Unsure if we need an explanation of this explicitly since it's already explained in the intro. Maybe just have it as an adjustable field in the final tool

In [ ]:
np.random.seed(42)
num_days = 60
initial_price = 100
strike_price = 100  
volatility = 0.3  
drift = 0.02  
dt = 1  

# Generate stock prices using Geometric Brownian Motion (GBM)
random_shocks = np.random.normal(drift * dt, volatility * np.sqrt(dt), num_days)
prices = initial_price * np.exp(np.cumsum(random_shocks))

# Label each day as ITM or OTM
def get_option_zone(price, strike):
    return "In The Money (ITM)" if price > strike else "Out of The Money (OTM)"

zones = [get_option_zone(p, strike_price) for p in prices]

# Create DataFrame for stock price
df_price = pd.DataFrame({
    "Day": np.arange(1, num_days + 1),
    "Stock Price": prices,
    "Zone": zones
})

# Create DataFrame for strike price for legend
df_strike = pd.DataFrame({
    "Day": np.arange(1, num_days + 1),
    "Stock Price": [strike_price] * num_days,
    "Zone": ["Strike Price"] * num_days  
})

# Combine both datasets
df_combined = pd.concat([df_price, df_strike])

# Color mapping for ITM, OTM, and Strike Price
color_scale = alt.Scale(domain=["In The Money (ITM)", "Out of The Money (OTM)", "Strike Price"], range=["green", "red", "blue"])

# Create stock price line chart with color regions
stock_chart = alt.Chart(df_combined).mark_line().encode(
    x=alt.X("Day:Q", title="Day"),
    y=alt.Y("Stock Price:Q", title="Stock Price"),
    color=alt.Color("Zone:N", scale=color_scale, legend=alt.Legend(title="Option Zone"))
).properties(
    width=700,
    height=400,
    title=" ITM or OTM Based on Current Stock Value and Strike Price"
)

# Display the chart
stock_chart.display()


# Drift (μ):
Drift represents the expected long-term trend of the price of a stock over time. It ignores short-term randomness (volatility) and reflects the average rate of return a stock is expected to achieve. 
- If drift is positive, a stock is expected to have an increase in value over time. 
- If drift is neutral, a stock is expected to have no change in value over time.
- If drift is negative, a stock is expected to have a decrease in value over time. 

The main purpose of drift in Monte Carlo simulations is to help simulate expected price movements over time.

TODO: Make it so the user can add more lines to the graph and move them around. When hovering over each line, it will grey out the other lines and tell them what the drift is of the line they are hovering over. 

In [ ]:
np.random.seed(42)
num_days = 60
initial_price = 100
volatility = 0.2  
dt = 1  
drift_0 = 0.0  
drift_10 = 0.10  

# Generate stock prices using Geometric Brownian Motion (GBM)
random_shocks = np.random.normal(0, volatility * np.sqrt(dt), num_days)
prices_drift_0 = initial_price * np.exp(np.cumsum(random_shocks))  # No drift
prices_drift_10 = initial_price * np.exp(np.cumsum(drift_10 * dt + random_shocks)) 

# Create DataFrame
df = pd.DataFrame({
    "Day": np.tile(np.arange(1, num_days + 1), 2),
    "Stock Price": np.concatenate([prices_drift_0, prices_drift_10]),
    "Drift": ["0% Drift"] * num_days + ["10% Drift"] * num_days
})

# Color mapping
color_scale = alt.Scale(domain=["0% Drift", "10% Drift"], range=["gray", "blue"])

# Create stock price line chart
stock_chart = alt.Chart(df).mark_line().encode(
    x=alt.X("Day:Q", title="Day"),
    y=alt.Y("Stock Price:Q", title="Stock Price"),
    color=alt.Color("Drift:N", scale=color_scale, legend=alt.Legend(title="Drift Level"))
).properties(
    width=700,
    height=400,
    title="0% Drift vs. 10% Drift on Same Stock"
)

stock_chart.display()


TODO: Give user access to full tool themselves with all variables and let them pick from 5 different options to track 

TODO: Decide which options to display. Maybe these?:
(All from different sectors: Technology: Apple, Finance: JPMorgan Chase, Healthcare: Pfizer, Energy: Chevron, Retail: Walmart)